In [ ]:
# Cell 1: Setup
import sys
import os
from pathlib import Path
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import yaml # For saving BoM

# Add scripts directory to path
repo_root = Path('.').resolve()
scripts_path = repo_root / 'scripts'
if str(scripts_path) not in sys.path:
    sys.path.append(str(scripts_path))

# Import project classes
from utils.config_loader import load_config
from inventory import Inventory
from bom import BoM, Component # Need Component for manual adds
from utils.api_clients import create_llm_client
# from scripts.utils.kicad_utils import LibraryManager # If doing pre-checks
# from scripts.check_assets import VerificationEngine # If doing pre-checks

# --- User Input ---
PROJECT_NAME = "GreenhouseController" # CHANGE THIS
# --- End User Input ---

project_dir = repo_root / PROJECT_NAME
project_bom_path = project_dir / "bom.yaml"

# Load Config and Initialize
try:
    config_obj = load_config()
    inventory = Inventory(repo_root / "inventory.yaml")
    inventory.load()
    llm = create_llm_client(config_obj)
    # lib_manager = LibraryManager(config_obj, project_dir) # Need project context
    # verifier = VerificationEngine(...) # If doing preliminary checks
    print("Setup complete. Inventory loaded.")
except Exception as e:
    print(f"Error during setup: {e}")




In [ ]:

# Cell 2: Define Project Goal
project_goal = """
Design a controller for a small greenhouse.
Requirements:
- Control 1 water pump (12V DC, ~1A) via relay.
- Control 3 solenoid valves (12V DC, ~0.5A each) via relay or driver IC.
- Control 2 linear actuators (+/-12V DC, ~2A stall current) for window vents.
- Read 3 analog soil moisture sensors (capacitive type, 5V).
- Read 1 DS18B20 temperature sensor (OneWire, 3.3V/5V).
- Use ESP32-DevKit-V1 as the main MCU.
- Power input: 12V DC barrel jack.
- Generate 5V and use ESP32's 3.3V.
"""
display(Markdown("### Project Goal"))
display(Markdown(project_goal))

In [ ]:

# Cell 3: Review Inventory
display(Markdown("### Available Inventory"))
display(inventory.to_dataframe())




In [ ]:
# Cell 4: LLM Circuit Suggestions
display(Markdown("### LLM Circuit Suggestions"))
inventory_summary = "\n".join([f"- {p.part_id}: {p.description} (Pkg: {p.package}, Footprint: {p.footprint})" for p in inventory.data.inventory_parts[:20]]) # Summarize inventory
prompt = f"Project Goal:\n{project_goal}\n\nRelevant Inventory Parts:\n{inventory_summary}\n\nSuggest key circuit blocks (Power, MCU, Solenoid Drivers, Actuator Drivers, Sensor Interfaces) and specific components for each. Prioritize using inventory parts where suitable (mention part_id). List connections conceptually. Identify parts definitely NOT in inventory."

suggestion_output = widgets.Output()
suggest_button = widgets.Button(description="Get LLM Suggestions")

def on_suggest_click(b):
     with suggestion_output:
          suggestion_output.clear_output()
          print("Querying LLM...")
          try:
               # Replace with actual LLM call using an appropriate method
               # suggestion = llm.generate_circuit_suggestion(prompt) # Assuming method exists
               suggestion = f"Placeholder LLM Suggestion for {PROJECT_NAME}:\n - Power: Use 12V input, buck converter (e.g., LM2596 module - NOT IN INVENTORY) for 5V.\n - MCU: ESP32-DevKit-V1 (Assume user provides).\n - Solenoids: Use ULN2003A (Part ID: INV005 if exists, otherwise NOT IN INVENTORY) driven by ESP32 GPIOs.\n - Actuators: Use TB6612FNG H-Bridge (NOT IN INVENTORY) driven by ESP32 GPIOs.\n - Sensors: Connect soil sensors to ADC pins, DS18B20 to GPIO with pull-up (INV001 if 4.7k Resistor exists)."
               display(Markdown(suggestion))
          except Exception as e:
               print(f"Error querying LLM: {e}")

suggest_button.on_click(on_suggest_click)
display(suggest_button, suggestion_output)



In [ ]:

# Cell 5: Interactive BoM Planning (Simplified Example)
display(Markdown("### Initial BoM Planning"))
# This would ideally use ipywidgets for a better UI
# For simplicity, we'll just define a dictionary based on suggestions/decisions
# User would edit this dictionary cell

initial_bom_dict = {
    'U1': {'value': 'ESP32-DevKit-V1', 'description': 'MCU Module', 'package': 'ESP32_DevKit', 'mpn': 'ESP32-DEVKITC-32D'},
    'U2': {'value': 'LM2596 Module', 'description': 'Buck Converter 5V', 'package': 'LM2596_MODULE'}, # Needs sourcing
    'U3': {'value': 'ULN2003A', 'description': 'Solenoid Driver', 'package': 'DIP-16', 'footprint': 'Package_DIP:DIP-16_W7.62mm', 'source_info': 'Inventory INV005'}, # Example using inventory
    'U4': {'value': 'TB6612FNG', 'description': 'H-Bridge Actuator Driver', 'package': 'SSOP-24'}, # Needs sourcing
    'R1': {'value': '4.7k', 'description': 'DS18B20 Pull-up', 'package': '0805', 'footprint': 'Resistor_SMD:R_0805_2012Metric', 'source_info': 'Inventory INV001'},
    # Add connectors, passives etc. based on design needs
    'J1': {'value': 'Barrel Jack', 'description': 'Power Input 12V', 'package': 'Barrel_Jack_Horiz'},
    # ...
}

In [ ]:

# Cell 6: Generate Initial bom.yaml
bom_list_for_yaml = []
for ref, data in initial_bom_dict.items():
     comp = Component(ref=ref, **data) # Use Pydantic model for structure
     bom_list_for_yaml.append(comp.model_dump(exclude_none=True))

bom_output_data = {'components': bom_list_for_yaml}

project_dir.mkdir(parents=True, exist_ok=True)
try:
    with open(project_bom_path, 'w') as f:
        yaml.dump(bom_output_data, f, sort_keys=False, indent=2)
    print(f"Initial BoM saved to: {project_bom_path}")
except Exception as e:
    print(f"Error saving initial BoM: {e}")

In [ ]:

# Cell 7: (Optional) Preliminary Asset Check & Area Estimate
# Add code here to run check_assets.py and calculate_fp_area.py on the generated bom.yaml
# This gives early feedback before detailed KiCad work.